In [13]:
import os
import shutil
import time
from multiprocessing import Pool, cpu_count

import cv2
import numpy as np


In [14]:
# Parameters

# files
folder_path_video = "video"
folder_path_frames_video_original= "frames_video_original"
folder_path_frames_video_result= "frames_video_result"

# video result
video_result_fps = 30 # frames por segundo

In [15]:
# ---------------------------
# 1. Limpiar el ambiente
# ---------------------------

for path in [folder_path_video, folder_path_frames_video_original, folder_path_frames_video_result]:
    if os.path.exists(path):
        shutil.rmtree(path)

os.makedirs(folder_path_video, exist_ok=True)
os.makedirs(folder_path_frames_video_original, exist_ok=True)
os.makedirs(folder_path_frames_video_result, exist_ok=True)

print("Carpetas creadas y ambiente limpio.")

Carpetas creadas y ambiente limpio.


In [16]:
# ---------------------------
# 2. Cargar el video local
# ---------------------------

video_source_path = "utils/cat_video.mp4"

if not os.path.exists(video_source_path):
    raise FileNotFoundError(f"No se encontró el video en {video_source_path}")

# Copiar el video a la carpeta 'video'
video_name = os.path.basename(video_source_path)
video_path = os.path.join(folder_path_video, video_name)
shutil.copy(video_source_path, video_path)

print(f"Video copiado a: {video_path}")

# Cargar video con OpenCV
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps if fps > 0 else 0

print(f"FPS: {fps:.2f} | Frames totales: {total_frames} | Duración: {duration:.2f} s")


Video copiado a: video/cat_video.mp4
FPS: 30.00 | Frames totales: 880 | Duración: 29.33 s


In [17]:
# ---------------------------
# 3. Obtener frames del video
# ---------------------------

# interval settings
seconds_interval = 1 / video_result_fps
step = int(video_result_fps * seconds_interval)

frame_idx = 0
saved_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Extraer solo los frames deseados
    if frame_idx % step == 0:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        output_path = os.path.join(folder_path_frames_video_original, f"frame_{saved_count:09d}.jpg")
        cv2.imwrite(output_path, cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))

        saved_count += 1

    frame_idx += 1

cap.release()

print(f"Procesados: {frame_idx:,} frames")
print(f"Guardados:  {saved_count:,} frames (1 cada {seconds_interval}s)")

Procesados: 880 frames
Guardados:  880 frames (1 cada 0.03333333333333333s)


In [18]:
# ================================================================
#   4A. Procesamiento SECUENCIAL de frames a escala de grises (OpenCV)
# ================================================================

# --- Limpiar carpeta de resultados antes de empezar ---
if os.path.exists(folder_path_frames_video_result):
    shutil.rmtree(folder_path_frames_video_result)
os.makedirs(folder_path_frames_video_result, exist_ok=True)

print("Procesando frames (SECUENCIAL con OpenCV)...")
start_time = time.time()

for i in range(saved_count):

    filename = f"frame_{i:09d}.jpg"
    input_path  = os.path.join(folder_path_frames_video_original, filename)
    output_path = os.path.join(folder_path_frames_video_result,   filename)

    bgr = cv2.imread(input_path)
    if bgr is None:
        print(f"No se encontró {filename}, se omite.")
        continue

    #Conversión rápida usando OpenCV
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    gray_bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    cv2.imwrite(output_path, gray_bgr)

end_time = time.time()
elapsed = end_time - start_time

print(f"Frames procesados: {saved_count}")
print(f"Tiempo total (SECUENCIAL): {elapsed:.4f} segundos")


Procesando frames (SECUENCIAL con OpenCV)...
Frames procesados: 880
Tiempo total (SECUENCIAL): 4.6825 segundos


In [19]:
# ================================================================
#   4B. Procesamiento PARALELO de frames a escala de grises (OpenCV)
# ================================================================

# --- Limpiar carpeta de resultados antes de procesar ---
if os.path.exists(folder_path_frames_video_result):
    shutil.rmtree(folder_path_frames_video_result)
os.makedirs(folder_path_frames_video_result, exist_ok=True)


def process_single_frame(i):
    filename = f"frame_{i:09d}.jpg"
    input_path  = os.path.join(folder_path_frames_video_original, filename)
    output_path = os.path.join(folder_path_frames_video_result,   filename)

    bgr = cv2.imread(input_path)
    if bgr is None:
        return f"⚠ {filename} saltado (no existe)"

    # Conversión rápida usando OpenCV
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    gray_bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    cv2.imwrite(output_path, gray_bgr)
    return f"✔ Procesado {filename}"


# --- Ejecutar en paralelo ---
num_cpus = cpu_count()
print(f"Procesando frames (PARALELO con {num_cpus} núcleos, OpenCV)...")

start_time = time.time()

with Pool(num_cpus) as pool:
    logs = pool.map(process_single_frame, range(saved_count))

end_time = time.time()
elapsed = end_time - start_time

print("\n".join(logs[:5]), "...")
print(f"Frames procesados: {saved_count}")
print(f"⏱️ Tiempo total (PARALELO): {elapsed:.4f} segundos")


Procesando frames (PARALELO con 8 núcleos, OpenCV)...
✔ Procesado frame_000000000.jpg
✔ Procesado frame_000000001.jpg
✔ Procesado frame_000000002.jpg
✔ Procesado frame_000000003.jpg
✔ Procesado frame_000000004.jpg ...
Frames procesados: 880
⏱️ Tiempo total (PARALELO): 1.3219 segundos


In [20]:
# ----------------------------------------------------------------
#   Ejemplo de Obtener cada imagen (después del procesamiento)
# ----------------------------------------------------------------

for i in range(saved_count):
    filename = f"frame_{i:09d}.jpg"
    file_path = os.path.join(folder_path_frames_video_result, filename)

    if os.path.exists(file_path):
        bgr_image = cv2.imread(file_path)
        rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
        print(f"imagen {i} leída: {filename} {rgb_image.shape}")
    else:
        print(f"⚠ Advertencia: No se encontró {filename} en {file_path}")


imagen 0 leída: frame_000000000.jpg (1024, 576, 3)
imagen 1 leída: frame_000000001.jpg (1024, 576, 3)
imagen 2 leída: frame_000000002.jpg (1024, 576, 3)
imagen 3 leída: frame_000000003.jpg (1024, 576, 3)
imagen 4 leída: frame_000000004.jpg (1024, 576, 3)
imagen 5 leída: frame_000000005.jpg (1024, 576, 3)
imagen 6 leída: frame_000000006.jpg (1024, 576, 3)
imagen 7 leída: frame_000000007.jpg (1024, 576, 3)
imagen 8 leída: frame_000000008.jpg (1024, 576, 3)
imagen 9 leída: frame_000000009.jpg (1024, 576, 3)
imagen 10 leída: frame_000000010.jpg (1024, 576, 3)
imagen 11 leída: frame_000000011.jpg (1024, 576, 3)
imagen 12 leída: frame_000000012.jpg (1024, 576, 3)
imagen 13 leída: frame_000000013.jpg (1024, 576, 3)
imagen 14 leída: frame_000000014.jpg (1024, 576, 3)
imagen 15 leída: frame_000000015.jpg (1024, 576, 3)
imagen 16 leída: frame_000000016.jpg (1024, 576, 3)
imagen 17 leída: frame_000000017.jpg (1024, 576, 3)
imagen 18 leída: frame_000000018.jpg (1024, 576, 3)
imagen 19 leída: frame

In [21]:
# ----------------------------------------------------------------
#   CREAR VIDEO A PARTIR DE LOS FRAMES PROCESADOS
# ----------------------------------------------------------------

import cv2
import os
import time

print("\nIniciando creación del video final...")

start_time = time.time()

# --- 1. Directorio de imágenes procesadas ---
folder_path = folder_path_frames_video_result

# --- 2. Nombre y ruta del video final ---
video_name = "video_result.mp4"
video_output_path = os.path.join(folder_path_video, video_name)

# --- 3. Validar existencia del primer frame ---
first_frame_path = os.path.join(folder_path, "frame_000000000.jpg")

if not os.path.exists(first_frame_path):
    raise FileNotFoundError(f"No se encontró el primer frame en {first_frame_path}")

# Leer primer frame para obtener resolución
frame = cv2.imread(first_frame_path)
height, width, layers = frame.shape
frame_size = (width, height)

# --- 4. Configurar VideoWriter ---
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_output_path, fourcc, video_result_fps, frame_size)

print(f"Creando video '{video_output_path}'")
print(f"Resolución: {width}x{height} | FPS: {video_result_fps}")
print(f"Frames a escribir: {saved_count}")

# --- 5. Iterar sobre los frames procesados ---
for i in range(saved_count):
    filename = f"frame_{i:09d}.jpg"
    file_path = os.path.join(folder_path, filename)

    img = cv2.imread(file_path)
    if img is not None:
        video.write(img)
    else:
        print(f"No se pudo leer {filename}, saltando...")

# --- 6. Guardar y cerrar ---
video.release()

end_time = time.time()
elapsed = end_time - start_time

print("\n¡Video generado exitosamente!")
print(f"Ruta del video final: {video_output_path}")
print(f"Tiempo total en construir el video: {elapsed:.4f} segundos\n")



Iniciando creación del video final...
Creando video 'video/video_result.mp4'
Resolución: 576x1024 | FPS: 30
Frames a escribir: 880

¡Video generado exitosamente!
Ruta del video final: video/video_result.mp4
Tiempo total en construir el video: 5.3744 segundos

